# Data Cleaning and Processing Script
### Imports

In [1]:
import numpy as np
import pandas as pd
import requests
import json
import os
import dotenv
import pylahman
import warnings
warnings.filterwarnings('ignore')


### Lahman Data pull in

In [2]:
# Pulls in all the relevant data from Lahman database
people = pylahman.People()
batting_stats = pylahman.Batting()
awards_file = pylahman.AwardsPlayers()
pitching = pylahman.Pitching()
fielding = pylahman.Fielding()
salaries = pylahman.Salaries()
allstar = pylahman.AllstarFull()

# get relevant awards
award_types = ['Cy Young Award', 'Most Valuable Player', 'Gold Glove', 'Silver Slugger', 'World Series MVP']
awards_filtered = awards_file[
    (awards_file['awardID'].isin(award_types)) & 
    (awards_file['yearID'] >= 2007) & 
    (awards_file['yearID'] <= 2017)
]

# filter allstar results
allstar = allstar[(allstar['yearID'] >= 2003) & (allstar['yearID'] <= 2017)]

### MLB Stats API Data pull in

In [3]:
# get directory of current file to access data
base_dir = os.getcwd()
data_dir = os.path.join(base_dir, '..', 'data')
filepath = os.path.join(data_dir, 'raw_data/MLB_stats_free_agents.csv')

free_agents = pd.read_csv(filepath)
free_agents.head()

# remove any minor-league contracts
free_agents = free_agents[
    free_agents['notes'].str.contains("contract", case=False, na=False) &
    ~free_agents['notes'].str.contains("Minor league", case=False, na=False)
].reset_index(drop=True)

# ensure data is from 2003 to 2015
free_agents = free_agents[free_agents['season'] < 2016]

### Clean and map Player ID to Player Names

In [4]:
from word2number import w2n
import re
from unidecode import unidecode

# create mapping of player to player id
player_dict = {}

# create mapping of player name to player id and clean spaces in names
for row in people.iterrows():
    s = str(row[1]['nameFirst'])
    s = s.replace(" ", "")
    player_dict[s + ' ' + str(row[1]['nameLast'])] = str(row[1]['playerID'])

contract_dict = {}

# clean accent marks
free_agents['player_full_name'] = free_agents['player_full_name'].apply(lambda x: unidecode(str(x)))

### Map Player ID to Free Agency Years and contract length

In [5]:

# gets each contract year mapped to player name with contract length
for _, row in free_agents.iterrows():
    name = unidecode(str(row['player_full_name']))
    
    # Remove suffixes and keep only first and last name
    # Remove common suffixes
    name = re.sub(r'\s+(Jr\.?|Sr\.?|III|II|IV)$', '', name, flags=re.IGNORECASE)
    
    # Split and keep only first and last (drops middle names too)
    name_parts = name.split()
    if len(name_parts) >= 2:
        name = name_parts[0] + ' ' + name_parts[-1]  # first + last
    
    year = str(row['season'])
    
    # Check if name exists in player_dict
    if name not in player_dict:
        continue  # Skip if player not found
    
    id = player_dict[name]

    # Extract and convert contract length
    contract_text = str(row['notes'])
    
    # Try to extract number (handles "10-year" or "10 year")
    number_match = re.search(r'(\d+)[-\s]year', contract_text)
    
    if number_match:
        contract_years = int(number_match.group(1))
    else:
        # Try word-based extraction (handles "One-year", "One year", etc.)
        word_match = re.search(r'(\w+)[-\s]year', contract_text, re.IGNORECASE)
        if word_match:
            first_word = word_match.group(1)
            try:
                contract_years = w2n.word_to_num(first_word)
            except:
                contract_years = None
        else:
            contract_years = None
    
    if id not in contract_dict:
        contract_dict[id] = []
    
    contract_dict[id].append((year, contract_years))


# Remove duplicate years while preserving order
for player_id in contract_dict:
    seen = set()
    unique_contracts = []
    for contract in contract_dict[player_id]:
        if contract not in seen:
            seen.add(contract)
            unique_contracts.append(contract)
    contract_dict[player_id] = unique_contracts


### Functions for Salary and Stats Aggregation

In [6]:
# gets pitching or batting stats for past 3 years
def find_stats(id, year, position):

    years_to_get = [year, year - 1, year - 2] 

    if position == 'P':  # Pitcher
        cols_to_sum = ['W', 'L', 'G', 'GS', 'CG', 'SHO', 'SV', 'IPOuts', 'H', 
                       'ER', 'HR', 'BB', 'SO', 'IBB', 'WP', 'HBP', 'BK', 
                       'BFP', 'GF', 'R', 'SH', 'SF', 'GIDP']
        cols_to_avg = ['ERA', 'BAOpp']  # ERA and Batting Average Against
        
        years_to_get = [year, year - 1, year - 2]
        
        df_3_year = pitching[
            (pitching['playerID'] == id) &
            (pitching['yearID'].isin(years_to_get))
        ]
        
        if df_3_year.empty:
            return None
        
        # Sum the counting stats
        sum_cols_present = [col for col in cols_to_sum if col in df_3_year.columns]
        summed = df_3_year[sum_cols_present].sum() if sum_cols_present else pd.Series()
        
        # Average ERA and BAOpp
        avg_cols_present = [col for col in cols_to_avg if col in df_3_year.columns]
        averaged = df_3_year[avg_cols_present].mean() if avg_cols_present else pd.Series()
        
        # Combine
        stats_3_year = pd.concat([summed, averaged]).to_frame().T
        
        return stats_3_year
    
    else: 
        cols_to_sum = ['G_batting', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 
                       'CS', 'BB', 'SO', 'IBB', 'HBP', 'SH', 'SF', 'GIDP']
        
        df_3_year = batting_stats[
            (batting_stats['playerID'] == id) &
            (batting_stats['yearID'].isin(years_to_get))
        ]
        
        if df_3_year.empty:
            return None
        
        # Sum all batting stats
        sum_cols_present = [col for col in cols_to_sum if col in df_3_year.columns]
        summed = df_3_year[sum_cols_present].sum() if sum_cols_present else pd.Series()
        
        stats_3_year = summed.to_frame().T
        
        return stats_3_year

# find position and fielding stats
def find_position(id, year):

    # Columns for operations
    cols_to_sum = ['InnOuts', 'PO', 'A', 'E', 'DP', 'PB', 'WP']
    avg_cols = ['ZR']
    years_to_get = [year, year - 1, year - 2]

    # 3-year fielding data
    df_3_year = fielding[
        (fielding['playerID'] == id) &
        (fielding['yearID'].isin(years_to_get))
    ]

    if df_3_year.empty:
        
        # Create a DataFrame with NaNs for sums/averages
        nan_data = {col: np.nan for col in cols_to_sum + avg_cols}
        year_3_totals = pd.DataFrame([nan_data])
        return np.nan, year_3_totals

    # Sum and average stats over 3 years
    sum_cols_present = [col for col in cols_to_sum if col in df_3_year.columns]
    avg_cols_present = [col for col in avg_cols if col in df_3_year.columns]

    summed_cols = df_3_year[sum_cols_present].sum() if sum_cols_present else pd.Series()
    zr_avg = df_3_year[avg_cols_present].mean() if avg_cols_present else pd.Series()

    year_3_totals = pd.concat([summed_cols, zr_avg]).to_frame().T

    # Determine primary position over 3 years
    primary_pos = df_3_year['POS'].mode()[0] if not df_3_year['POS'].mode().empty else np.nan

    return primary_pos, year_3_totals

# find current age of player at free agency
def find_age(id, year):
    born = people[people["playerID"] == id]
    year_born = born["birthYear"]
    return year - year_born

# create unique id
def create_id(id, year):
    return str(id) + "_" + str(year)

# returns average salary over the contract length starting the year after signing   
def find_salary(player_id, year, contract_length):

    # Validate inputs
    if pd.isna(year) or pd.isna(contract_length):
        return np.nan

    try:
        year = int(year)
        contract_length = int(contract_length)
    except:
        return np.nan

    collected = []

    # Loop through each year covered by the contract
    for i in range(1, contract_length + 1):
        next_year = year + i

        # Data cutoff
        if next_year > 2016:
            break

        row = salaries[
            (salaries['playerID'] == player_id) &
            (salaries['yearID'] == next_year)
        ]

        if not row.empty:
            salary_val = row['salary'].iloc[0]
            if pd.notna(salary_val):
                collected.append(salary_val)

    return np.mean(collected) if collected else np.nan

# finds contract length
def find_contract_length(player_id, year):
    
    # Validate year
    if pd.isna(year):
        return np.nan

    try:
        year = int(year)
    except:
        return np.nan

    # Player must be in contract dictionary
    if player_id not in contract_dict:
        return np.nan

    contracts = contract_dict[player_id]

    # Match contract by year (ensure type consistency)
    for yr, length in contracts:
        if int(yr) == year:
            # Validate contract length
            if pd.isna(length):
                return np.nan
            try:
                return int(length)
            except:
                return np.nan

    return np.nan

# gets awards for each
def find_awards(player_id, year):
    years_to_check = [year, year - 1, year - 2]
    
    # Filter awards for this player in past 3 years
    player_awards = awards_filtered[
        (awards_filtered['playerID'] == player_id) &
        (awards_filtered['yearID'].isin(years_to_check))
    ]
    
    # Filter All-Star appearances for this player in past 3 years
    allstar_appearances = allstar[
        (allstar['playerID'] == player_id) &
        (allstar['yearID'].isin(years_to_check))
    ]
    
    # Create binary columns for each award type
    award_cols = {
        'won_cy_young': 0,
        'won_mvp': 0,
        'won_gold_glove': 0,
        'won_silver_slugger': 0,
        'all_star': 0
    }
    
    # Check which awards they won
    if not player_awards.empty:
        awards_won = player_awards['awardID'].unique()
        
        if 'Cy Young' in awards_won or 'Cy Young Award' in awards_won:
            award_cols['won_cy_young'] = 1
        if 'MVP' in awards_won or 'Most Valuable Player' in awards_won:
            award_cols['won_mvp'] = 1
        if 'Gold Glove' in awards_won:
            award_cols['won_gold_glove'] = 1
        if 'Silver Slugger' in awards_won:
            award_cols['won_silver_slugger'] = 1
    
    # Check All-Star appearances
    if not allstar_appearances.empty:
        award_cols['all_star'] = 1
    
    return award_cols

# find the average salary for the year
def find_avg_sal(year):
    sal = salaries[(salaries['yearID'] == year)]
    avg_sal = sal['salary'].mean()
    return avg_sal



### Data Processing and Output 

In [7]:

# lists to store data
final_pitchers = []
final_batters = []

# diagnostic variables
no_matches = 0
matches = 0

# loop through free agents and aggregate stats and salary data
for _, row in free_agents.iterrows():
    
    if row['player_full_name'] in player_dict:
        matches += 1
        
        # --- Basic attributes ---
        year = row['season']
        player_id = player_dict[row['player_full_name']]
        awards = find_awards(player_id, year)
        age = int(find_age(player_id, year))
        row_id = create_id(player_id, year)
        
        # --- Salaries ---
        avg_sal = find_avg_sal(year)
        contract_length = find_contract_length(player_id, year)
        sal = find_salary(player_id, year, contract_length)

        # --- Stats ---
        pos, field_stats = find_position(player_id, year)

        stats_3yr = find_stats(player_id, year, pos)
        # Skip players without stats
        if stats_3yr is None or field_stats is None:
            continue
        
        # Create rows of data for pitchers and batters
        # PITCHER ROW
        if pos == "P":

            df_row = pd.concat([stats_3yr, field_stats], axis=1)

            df_row.insert(0, "row_id", row_id)
            df_row.insert(1, "playerID", player_id)
            df_row.insert(2, "year", year)
            df_row.insert(3, "position", pos)
            df_row.insert(4, "age", age)
            df_row.insert(5, "avg_salary_year", avg_sal)
            df_row.insert(6, "free_agent_salary", sal)
            df_row.insert(7, "contract_length", contract_length)

            # Add award columns
            for k, v in awards.items():
                df_row[k] = v

            final_pitchers.append(df_row)

        # Batter ROW
        else:

            df_row = pd.concat([stats_3yr, field_stats], axis=1)

            df_row.insert(0, "row_id", row_id)
            df_row.insert(1, "playerID", player_id)
            df_row.insert(2, "year", year)
            df_row.insert(3, "position", pos)
            df_row.insert(4, "age", age)
            df_row.insert(5, "avg_salary_year", avg_sal)
            df_row.insert(6, "free_agent_salary", sal)
            df_row.insert(7, "contract_length", contract_length)

            # Add award columns
            for k, v in awards.items():
                df_row[k] = v

            final_batters.append(df_row)

    else:
        no_matches += 1

# convert pitchers list to dataframe
if final_pitchers:
    final_pitchers_df = pd.concat(final_pitchers, ignore_index=True)
else:
    final_pitchers_df = pd.DataFrame()

# convert batters list to dataframe
if final_batters:
    final_batters_df = pd.concat(final_batters, ignore_index=True)
else:
    final_batters_df = pd.DataFrame()

# output to csv
final_pitchers_df.to_csv(data_dir + '/cleaned/final_pitchers_df.csv', index=False)
final_batters_df.to_csv(data_dir + '/cleaned/final_batters_df.csv', index=False)

# print diagonistic variables
print(f"Number of non-matches: {no_matches}")
print(f"Number of matches: {matches}")


Number of non-matches: 33
Number of matches: 1099


In [9]:
# save off lahman data into csv

batting_stats.to_csv(data_dir + '/raw_data/lahman_data/batting_stats.csv', index=False)
pitching.to_csv(data_dir + '/raw_data/lahman_data/pitching.csv', index=False)
fielding.to_csv(data_dir + '/raw_data/lahman_data/fielding.csv', index=False)
salaries.to_csv(data_dir + '/raw_data/lahman_data/salaries.csv', index=False)
awards_file.to_csv(data_dir + '/raw_data/lahman_data/awards.csv', index=False)
allstar.to_csv(data_dir + '/raw_data/lahman_data/allstar.csv', index=False)
people.to_csv(data_dir + '/raw_data/lahman_data/people.csv', index=False)